In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates, MSMultipleDates, PIMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential2

from src.manager.evidential import ManagerEvidential
from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.manager.evidential as evidential
from src.Logger import Logger
import pdb
import gc


Tensorflow ver. 2.6.0


In [3]:

with open('config/config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: evidential2. Site: MS. Training: False. Training date: earlier
{'training': False, 'inferring': True, 'site': 'MS', 'training_date': 'earlier', 'mode': 'evidential2', 'training_times': 10, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 39, 'get_multiple_metrics': False, 'start_repetition_id': 4, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': False, 'log_filename': 'log.pkl'}


In [4]:

if config['site'] == 'PA':
	if config['training_date'] == 'earlier':
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)	

if config['mode'] == 'ensemble':
	if config['training'] == True:
		config['mode'] = 'single_run'
	else:
		sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

if config['training'] == False:
	if config['site'] == 'PA':
		if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
			if config['training_date'] == 'current': exp = 0
			elif config['training_date'] == 'earlier': exp = 1
				
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 8; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 9; repetition_id = 0

	elif config['site'] == 'MT':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 2
			elif config['training_date'] == 'earlier': exp = 3
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 10; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 11; repetition_id = 0

	elif config['site'] == 'MS':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 4
			elif config['training_date'] == 'earlier': 
				exp = 37

		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 12; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0
	elif config['site'] == 'PI':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 6
			elif config['training_date'] == 'earlier': exp = 27
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 14; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0

else:
	exp = config['training_experiment_id']
ic(dates)


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [5]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [6]:
exp

39

In [7]:
logger = Logger()


In [8]:
if config['training'] == True:
    

    manager = manager_class(config, dataset, patchesHandler, logger)
    print("manager.config['dropout_training']", manager.config['dropout_training'])
    manager.defineExperiment(exp) # fixed
    
    manager.setExperimentPath()
    manager.createLogFolders()

    manager.loadDataset()

    # %%
    if config["training"] == True:
        manager.run()


In [9]:



if config['site'] == 'PA':
	dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)


ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()


ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [10]:
manager_class

src.manager.multioutput.ManagerEvidential2

In [11]:
exp

39

In [12]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


repetition_ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], repetition_n 10


In [13]:
config['start_repetition_id']

4

In [14]:
if config['inferring'] == True:
    results = []
    error_count = 0

    # while idx < len(repetition_n):
    for idx in range(config['start_repetition_id'], repetition_n): # 10 repetitions
    # for idx in range(0, 4): # 10 repetitions


        print("Beginning run number {}".format(idx))
        logger = Logger()
        manager = manager_class(config, dataset, patchesHandler, logger)

        print("manager.config", manager.config)

        manager.defineExperiment(exp) # fixed
        manager.defineRepetitionId(idx) # varying from 0 to 10
        manager.setExperimentPath()
        manager.createLogFolders()

        manager.loadDataset()

        # %%
        if config['get_multiple_metrics'] == False:
            predictor_fcn = manager.run_predictor
        else:
            predictor_fcn = manager.run_predictor_multiple_metrics
        result = predictor_fcn()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)

        with open(manager.default_log_name, 'wb') as f:
            pickle.dump(results, f)
        print("Grid execution idx: {}".format(idx))
        del manager
        gc.collect()
        # idx += 1
    

Beginning run number 4
manager.config {'training': False, 'inferring': True, 'site': 'MS', 'training_date': 'earlier', 'mode': 'evidential2', 'training_times': 10, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 39, 'get_multiple_metrics': False, 'start_repetition_id': 4, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': False, 'log_filename': 'log.pkl', 'inference_times': 1, 'dropout_training': False}
D:/Jorge/datasets/deforestation/MS/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([334024764,   8717093,   6541094,   3824922,   2096535,   1305948,
                                                                      1003721,    903906,    318788,    701575,    310887,    440417,
                                                                       316180,    560438,    600838,    776899], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([334024764,   8717093,   6541094,   3824922,   2096535,   1305948,
                                                                      1003721,    903906,    318788,    701575,    310887,    440417,
                                                                       316180,    560438,    600838,    776899], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

ic| image_stack.shape: (18605, 19481, 9)


Image shape:  (18605, 19481, 9)
mask:  (18605, 19480)
image stack:  (18605, 19480, 9)
ref : (18605, 19480, 1)
Training tiles:  [2, 4, 5, 6, 7, 12, 14, 15, 18, 21, 23, 24]
Validation tiles:  [9, 11, 25]
Test tiles:  [1, 3, 8, 10, 13, 16, 17, 19, 20, 22]
-35.0 -8.0


ic| self.path_models+ '/' + self.method +'_'+str(self.repetition_id)+'.h5': 'D:/Jorge/datasets/deforestation/experiments/MS/exp39/models/resunet_7.h5'


Dropout training mode: False
time:  0
alpha.shape (18640, 19488, 2)
S.shape (18640, 19488)
K 2
u.shape (18640, 19488)
belief.shape (18640, 19488, 2)
Inference runtime 140.62
(36877090,)
(array([0, 1], dtype=uint8), array([36581631,   295459], dtype=int64))
(18605, 19480, 2)


ic| predicted_test.shape: (144970160,)
ic| predicted_test.shape: (36877090,)


In [ ]:
error_count

0

In [ ]:
print(results)

[{'uncertainty_result': {'metrics': {'precision_L': array([0.81588878]), 'recall_L': array([0.90360463]), 'recall_Ltotal': array([0.7227319]), 'AA': array([0.02999761]), 'precision_H': array([0.24574504]), 'recall_H': array([0.53130907]), 'UEO': array([0.10305995]), 'f1_L': array([0.8575094]), 'f1_H': array([0.33605528]), 'f1': 71.52}, 'metrics_audited': {'precision': array([0.84982389]), 'recall': array([0.9228999]), 'f1': array([0.88485571])}, 'exp': 38}}, {'uncertainty_result': {'metrics': {'precision_L': array([nan]), 'recall_L': array([0.]), 'recall_Ltotal': array([0.]), 'AA': array([0.99999965]), 'precision_H': array([nan]), 'recall_H': array([0.]), 'UEO': array([0.00801175]), 'f1_L': array([nan]), 'f1_H': array([nan]), 'f1': 0.0}, 'metrics_audited': {'precision': array([1.]), 'recall': array([0.99996954]), 'f1': array([0.99998477])}, 'exp': 38}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.90079848]), 'recall_L': array([0.87851248]), 'recall_Ltotal': array([0.823

In [ ]:
print(len(results))

5


In [ ]:
results[7]

IndexError: list index out of range